In [ ]:
import pandas as pd
import numpy as np
import requests
import json

In [ ]:
key="<MY TOKEN>"

In [ ]:
users=[]
def github(x):
  url = "https://api.github.com/search/users"
  headers = {
      "Accept": "application/vnd.github+json",
      "Authorization": f"Bearer {key}",
      "X-GitHub-Api-Version": "2022-11-28"
  }
  payload={
      "q":"location:Sydney followers:>100",
      "sort":"followers",
      "per_page":100,
      "page":x
  }
  response=requests.get(url,headers=headers,params=payload)
  return response.json()["items"]

In [ ]:
for i in range(1,5):
  users+=github(i)

In [ ]:
users[0]

In [ ]:
headers = {
      "Accept": "application/vnd.github+json",
      "Authorization": f"Bearer {key}",
      "X-GitHub-Api-Version": "2022-11-28"
}
cleaned_users=[]
for user in users:
  url=user["url"]
  response=requests.get(url,headers=headers)
  cleaned_users.append(response.json())

In [ ]:
def get_repos(x,user):
  url=user["repos_url"]
  headers = {
      "Accept": "application/vnd.github+json",
      "Authorization": f"Bearer {key}",
      "X-GitHub-Api-Version": "2022-11-28"
  }
  payload={
      "sort":"pushed",
      "direction":"desc",
      "per_page":100,
      "page":x
  }
  response=requests.get(url,headers=headers,params=payload)
  return response


In [ ]:
headers = {
      "Accept": "application/vnd.github+json",
      "Authorization": f"Bearer {key}",
      "X-GitHub-Api-Version": "2022-11-28"
}
repos=[]
for user in users:
  repo=[]
  page=1
  data=get_repos(page,user)
  while data.json() and len(repo)<500:
    data=get_repos(page,user)
    repo+=data.json()
    page+=1
  repos+=repo

In [ ]:
def repo_extractor(repo):
  login=str(repo['owner']['login'])
  fullname=str(repo['full_name'])
  created_at=str(repo['created_at'])
  stargazers_count=str(repo['stargazers_count'])
  watchers_count=str(repo['watchers_count'])
  language=str(repo['language'])
  has_projects=str(repo['has_projects'])
  has_wiki=str(repo['has_wiki'])
  if repo['license'] is not None:
    license_name=repo['license']['key']
  else:
    license_name="''"
  data=[login,fullname,created_at,stargazers_count,watchers_count,language,has_projects,has_wiki,license_name]
  return data

In [ ]:
text=[]
for repo in repos:
  text.append(",".join(repo_extractor(repo)))

In [ ]:
text_processed=list(map(lambda x:x+"\n",text))

In [ ]:
repos_file=open("repos.csv","w")
repos_file.writelines(text_processed)
repos_file.close()

In [ ]:
def users_extractor(user):
  login=user['login'].replace(",",";")
  name=user['name'].replace(",",";") if user["name"] else "''"
  company=user['company'].strip().upper().replace("\n","").replace("\r","").replace(",",";") if user['company'] else "''"
  location=user['location'].replace(",",";") if user["location"] else "''"
  email=user['email'].replace(",",";") if user['email'] else "''"
  hireable=str(user['hireable']).replace(",",";") if user['hireable'] else "''"
  bio=user['bio'].replace("\n","").replace("\r","").replace(",",";") if user['bio'] else r"''"
  public_repos=str(user['public_repos']).replace(",",";")
  followers=str(user['followers']).replace(",",";")
  following=str(user['following']).replace(",",";")
  created_at=str(user['created_at']).replace(",",";")
  return [login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at]

In [ ]:
text=[]
for user in cleaned_users:
  text.append(",".join(users_extractor(user)))
text_processed=list(map(lambda x:x+"\n",text))
users_file=open("users.csv","w",encoding="utf-8")
users_file.writelines(text_processed)
users_file.close()